# 5章 了解最佳化問題的全貌
本章要透過排班問題學習各種解決最佳化問題的方法。

In [ ]:
#Colaboratory環境的設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter5

In [ ]:
#設定函式庫
!pip install -q -r ./requirements.txt

## 5-2 試著利用求解器解決線性最佳化問題

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals
from IPython.display import display

# 載入資料
df_n = pd.read_csv('nutrition.csv', index_col="食品")
df_p = pd.read_csv('price.csv')
print("食品與營養素的關係")
display(df_n)
print("食品的價格")
display(df_p)

# 初始設定 #
np.random.seed(1)
np = len(df_n.index)
nn = len(df_n.columns)
pr = list(range(np))

# 建立數理模型 #
m1 = model_min()
# 目標函數
v1 = {(i):LpVariable('v%d'%(i),cat='Integer',lowBound=0) for i in pr}
# 條件
m1 += lpSum(df_p.iloc[0][i]*v1[i] for i in pr)
for j in range(nn):
    m1 += lpSum(v1[i]*df_n.iloc[i][j] for i in range(np)) >= 100
m1.solve()

# 計算總成本 #
print("最佳解")
total_cost = 0
for k,x in v1.items():
    i = k
    print(df_n.index[i],"的個數:",int(value(x)),"個")
    total_cost += df_p.iloc[0][i]*value(x)

print("總成本:",int(total_cost),"元")

## 5-3. 試著解決非線性最佳化問題 

### 利用二元搜尋演算法計算1000的平方根

In [ ]:
def f(x):
    return x**2 - 1000 

# 初始設定
lo = -0.1
hi = 1000.1
eps = 1e-10 # 容許誤差

# 執行二元搜尋演算法
count = 0
while hi-lo > eps:
    x = (lo + hi) / 2
    if f(x) >= 0:
        hi = x 
    else:
        lo = x 
    count += 1

print(f'結果: {hi}')
print(f'搜尋次數: {count}次')

### 利用牛頓法計算1000的平方根

In [ ]:
# 牛頓法的函數
# x0, eps為預設值
def square_root(y, x0=1, eps=1e-10):
    x = x0
    count = 0
    while abs(x**2 - y) > eps:
        x -= (x*x - y) / (2*x)
        count += 1
    return x, count

# 執行牛頓法
x, count = square_root(1000)
print(f'結果: {x}')
print(f'搜尋次數: {count}次')

## 5-4. 試著設計自動安排鐘點員工班表的方法

### 載入排班意願表

In [ ]:
import pandas as pd 
def schedules_from_csv(path):
    return pd.read_csv(path, index_col=0)

schedules_from_csv('schedule.csv')

## 5-5. 試著利用Graph Network可視化排班意願

In [ ]:
import numpy as np

print(f'隨機產生100個1 ~ 100的整數')
a = np.random.randint(1, 100, 100)
print(a)

l = list(a)
s = set(a)

x = 50

# 下列2行程式會得到相同的結果
print(f'是否包含50這個數字: {x in l}')
print(f'是否包含50這個數字: {x in s}')

In [ ]:
%%time
for _ in range(10**6):
    x in l # 以list確認的情況

In [ ]:
%%time 
for _ in range(10**6):
    x in s # 以set確認的情況

## 5-9 試著解決最大流問題

### 計算最大流問題的程式碼

In [ ]:
from collections import deque
inf = float('inf')

class Graph:
    class __Edge:
        def __init__(self, capacity=1, **args):
            self.capacity = capacity

            
    def __init__(self, n=0):
        self.__N = n
        self.edges = [{} for _ in range(n)]

    
    # 追加邊的函數
    def add_edge(self, u, v, **args): 
        self.edges[u][v] = self.__Edge(**args)

    
    # 執行BFS(寬度優先搜尋)的函數
    def bfs(self, src=0):
        n = self.__N
        self.lv = lv = [None]*n
        lv[src] = 0
        q = deque([src]) # BFS使用了queue這種資料結構(Python則是使用dequeue)
        while q:
            u = q.popleft()
            for v, e in self.edges[u].items():
                if e.capacity == 0: continue # 無法讓水流過去(沒有邊)
                if lv[v] is not None: continue # 層級已確定
                lv[v] = lv[u] + 1
                q.append(v)
    
    # 執行DFS(深度優先搜尋)的函數
    def flow_to_sink(self, u, flow_in, sink):
        if u == sink:
            return flow_in
        flow = 0
        for v, e in self.edges[u].items():
            if e.capacity == 0: continue
            if self.lv[v] <= self.lv[u]: continue 
            f = self.flow_to_sink(v, min(flow_in, e.capacity), sink)
            if not f: continue
            self.edges[u][v].capacity -= f
            if u in self.edges[v]:
                self.edges[v][u].capacity += f
            else:
                self.add_edge(v, u, capacity=f)
            flow_in -= f
            flow += f
        return flow

    
    # 不斷執行BFS與DFS，直到求出最大水流為止
    def dinic(self, src, sink, visualize=False):
        flow = 0
        while True:
            if visualize:
                self.visualizer(self)
            self.bfs(src)
            if self.lv[sink] is None:
                return flow
            flow += self.flow_to_sink(src, inf, sink)
            
    # 設定可視化函數
    def set_visualizer(self, visualizer):
        self.visualizer = visualizer

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,5))

# 設定邊
edges = [
    ((0, 2), 3),
    ((0, 1), 9),
    ((1, 2), 8),
    ((2, 3), 7),
    ((1, 4), 2),
    ((2, 5), 5),
    ((3, 4), 6),
    ((4, 5), 4),
    ((4, 6), 3),
    ((5, 6), 10)
]

# 設定頂點座標
nodes = [
    (0, 1),
    (1, 0),
    (1, 2),
    (2, 1),
    (3, 0),
    (3, 2),
    (4, 1),
]

n = len(nodes)

# 可視化使用的圖表
graph = nx.DiGraph()

# 在圖表追加頂點編號
graph.add_nodes_from(range(n))

# 將頂點座標的資訊整理成方便在圖表新增的格式
pos = dict(enumerate(nodes))

# 繪製最初的狀態
plt.figure(figsize=(10,5))

for (u, v), cap in edges:
    graph.add_edge(u, v, capacity=cap)

labels = nx.get_edge_attributes(graph,'capacity')
nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels, font_color='r', font_size=20)
nx.draw_networkx(graph, pos=pos, node_color='c')
plt.show()
graph.remove_edges_from([e[0] for e in edges])

In [ ]:
# 產生計算最大水流的圖表
g = Graph(n)

for (u, v), cap in edges: 
    g.add_edge(u, v, capacity=cap) # 追加邊。

    
# 繪製中途水流情況的函數
def show_progress(g):
    plt.figure(figsize=(10,5))

    for (u, v), cap in edges:
        e = g.edges[u][v]
        if e.capacity >= cap:
            continue 
        graph.add_edge(u, v, capacity=cap-e.capacity)

    labels = nx.get_edge_attributes(graph,'capacity')
    nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels, font_color='g', font_size=20)
    nx.draw_networkx(graph, pos=pos, node_color='c')
    plt.show()
    graph.remove_edges_from([e[0] for e in edges])

# 設定可視化函數
g.set_visualizer(show_progress)

print(f'最大水流: {g.dinic(src=0, sink=6, visualize=True)}')

## 5-10.試著利用最大流問題的解法解決排班問題

### 載入班表

In [ ]:
import numpy as np 

import pandas as pd 
def schedules_from_csv(path):
    return pd.read_csv(path, index_col=0)

schedules = schedules_from_csv('schedule.csv')
n, m = schedules.shape
schedules

### 將排班意願表轉換成network

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 10))

# 可視化使用的圖表
graph = nx.DiGraph()

N = n + m + 2 
graph.add_nodes_from(range(N))
# 在圖表追加n個頂點
center = 10
vertices = [(center,9)] + [(center + (i-n//2), 6) for i in range(n)] + [(center+ (i-m//2), 3) for i in range(m)] + [(center, 0)]


# 建立邊
schedules = schedules.values
edges = np.argwhere(schedules)
edges += 1
edges[:,1] += n
edges1 = np.array([(0, i+1) for i in range(n)]).reshape(-1, 2)
edges2 = np.array([(i+n+1, n+m+1) for i in range(m)]).reshape(-1, 2)
edges = np.vstack([edges1, edges, edges2])

# 將頂點座標的資訊整理成容易於圖表新增的格式
pos = dict(enumerate(vertices))

# 追加邊
for u, v in edges:
    graph.add_edge(u, v, capacity=1)

# 繪製圖表
nx.draw_networkx(graph, pos=pos, node_color='c') 
plt.show()

### 計算最大流問題的最佳解

In [ ]:
g = Graph(N)

# 追加邊
for u, v in edges: 
    g.add_edge(u, v, capacity=1)

print(f'最大水流: {g.dinic(src=0, sink=N-1)}')

### 畫出結果

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 10))

# 可視化使用的圖表
graph = nx.DiGraph()

N = n + m + 2 
graph.add_nodes_from(range(N))
center = 10

# 決定繪圖的座標
vertices = [(center,9)] + [(center + (i-n//2), 6) for i in range(n)] + [(center+ (i-m//2), 3) for i in range(m)] + [(center, 0)]

# 建立邊
edges = np.argwhere(schedules)
edges += 1
edges[:,1] += n

# 將頂點座標的資訊整理成適合新增至圖表的格式
pos = dict(enumerate(vertices))

# 初始化班表
shift_table = np.zeros(shape=(n, m), dtype=np.int8)

# 追加邊
for u, v in edges:
    e = g.edges[u][v]
    if e.capacity == 1:# 不繪製還沒配對的邊
        continue
    graph.add_edge(u, v, capacity=1)
    u -= 1 # 轉換成鐘點員工的index
    v -= 1 + n # 轉換成班表格子的index
    shift_table[u, v] = 1

# 繪製圖表
nx.draw_networkx(graph, pos=pos, node_color='c')
plt.show()

### 輸出最終的班表

In [ ]:
# 轉換成資料框架
shift_table = pd.DataFrame(shift_table)

# 設定欄位與索引
idx = [f'鐘點員工{i}' for i in range(n)]
col = [
    f'{day}{time}'
    for day in ['星期一', '星期二', '星期三', '星期四', '星期五']
    for time in ['早上', '中午', '晚上']
]
shift_table.rename(index=dict(enumerate(idx)), columns=dict(enumerate(col)))